In [1]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import RFormula
import numpy as np
import time

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

Primero se lee la base guardada previamente en el notebook de dask

In [3]:
data = spark.read.csv("0.part", header =True, inferSchema = True)

data = data.drop("_c0")
data.show()

+-----------+---------------+-------------+-------------------+----------+----------+
|fare_amount|passenger_count|trip_distance|                  y|car_type_A|car_type_B|
+-----------+---------------+-------------+-------------------+----------+----------+
|       22.0|              1|          6.9|0.20909090909090908|         1|         0|
|        9.0|              1|         1.81|                0.0|         1|         0|
|        7.5|              1|         0.96|0.13333333333333333|         1|         0|
|        8.5|              1|          1.9|0.11764705882352941|         1|         0|
|        7.5|              1|          1.0|0.22133333333333333|         1|         0|
|        9.5|              5|         1.71|0.15789473684210525|         1|         0|
|        8.0|              1|         1.27|             0.1875|         1|         0|
|        7.5|              4|         1.55|0.21333333333333335|         1|         0|
|        6.0|              5|         0.54|           

Se crea la formula de la regresion (para no tener que generar las variables label y features manualmente)

In [4]:
f = RFormula(formula = "y ~ .")
data = f.fit(data).transform(data)

Se crea el grid

In [5]:
ridge = LinearRegression(elasticNetParam=0.0)

paramGrid = ParamGridBuilder().addGrid(ridge.regParam,[0.01, 0.03, 0.05, 0.07, 0.09, 0.1]).build()

evaluator = RegressionEvaluator(metricName='mse')
cv = CrossValidator(estimatorParamMaps=paramGrid,
                   estimator = ridge,
                   evaluator = evaluator,
                   numFolds = 10,
                   parallelism = 4)

Se ejecuta el grid

In [6]:
%%time
ridge_cv = cv.fit(data)

CPU times: user 2.99 s, sys: 955 ms, total: 3.95 s
Wall time: 20.9 s


In [7]:
print("Loss: %f"%(np.min(ridge_cv.avgMetrics)))

Loss: 0.016302
